In [27]:
profile_output = "GCA_000013405.1_ASM1340v1_profile_output"
#"GCA_000011185.1_ASM1118v1_profile_output"
lookup = { 0:'A', 1:'C', 2:'G', 3:'T'}
found_matrix = False 
found_start = False
found_end = False
SDs = []
SD = ""
with open(profile_output, "r+") as f:
    for line in f:
        if (line.startswith("% Weight matrix composed")):
            found_matrix = True
        elif (line.startswith("     A")):
            pass
        elif (found_matrix):
            numbers = line.strip().split("\t")
            if (len(numbers) != 6) :
                continue # bug in output
            if (not found_start and float(numbers[5]) >= 0.4): #0.35): #0.4
                found_start = True
                found_end = False
                SD = ""
                #print("found start")
            if (found_start and (float(numbers[5]) < 0.4)): #0.35)): #0.4
                found_end = True
                found_start = False
                #print("found end")
                SDs.append(SD)
            if (found_start and not found_end):
                max_num = float(numbers[0])
                max_i = 0
                for i in range(1,4):
                    if (float(numbers[i]) > max_num):
                        max_num = float(numbers[i])
                        max_i = i
                SD = SD + lookup[max_i]
maxSD = ""
for SD in SDs:
    if(len(SD) > len(maxSD)):
        maxSD = SD
print(SDs)
print(maxSD)

['AGGAG']
AGGAG


In [12]:
# This is an OLD VERSION, see cell below for new one
#### FOR TESTING, remove keep only valid orfs from Tompa search
from subprocess import Popen, PIPE, STDOUT
import subprocess
import csv
import scipy
from os.path import join

# expected perl: v5
# expected input files:
# / *_genomic.fna.gz
# keep_only_valid_orfs.pl, tompa_test installed, 

def run_Tompa_pipeline(file_prefix, input_path, output_path): 

    print(file_prefix)
    rbs_input = output_path + file_prefix + '_RBSidentify_input'
    rbs_output = output_path + file_prefix + '_RBSidentify_output'
    profile_input = output_path + file_prefix + '_profile_input'
    profile_output = output_path + file_prefix + '_profile_output' 
    
    # pre-process to get rid of bad orfs
    cds_file = input_path + file_prefix + "_cds_from_genomic.fna.gz"
    processed_cds_file = output_path + file_prefix + "_cds_processed.fa"
    
    #p = Popen("gunzip" + " -c " + cds_file + " > " + processed_cds_file, shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    #ret = p.communicate()[0]
    try:
        ret = subprocess.check_output(["perl", "keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing keep_only_valid_orfs for ", cds_file, "; ", exc.returncode, exc.output)
    
    #TODO later make sequence upstream of genes handle multiple chromosomes!!!!    
    full_genome_file = input_path + file_prefix + "_genomic.fna.gz"
    try:
        ret = subprocess.check_output(["perl", "get_sequence_upstream_of_gene.pl", processed_cds_file,\
                                      full_genome_file, rbs_input], stderr=subprocess.STDOUT)
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing get_sequence_upstream_of_gene.pl for ", cds_file, "; ", exc.returncode, exc.output)
    
    
    # run RBSidentify
    p = Popen(['tompa_test/RBSidentify/RBSidentify'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input= (rbs_input + '\n' + rbs_output + '\nn\n7\n1\nn\n').encode())[0]

    
    # process output of RBSidentify 
    p = Popen("tail -n +27 " + rbs_output  + "|  awk '{print $NF,$0}' | sort -nr | cut -f2- -d' ' >"  + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()

    # run profile
    p = Popen(['tompa_test/profile/profile'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input=(profile_input + '\n' + rbs_input + '\n'  + profile_output + '\n20\n7\n1\nn\n').encode())[0]

    lookup = { 0:'A', 1:'C', 2:'G', 3:'T'}
    found_matrix = False 
    found_start = False
    found_end = False
    SD = ""
    with open(profile_output, "r+") as f:
        for line in f:
            if (line.startswith("% Weight matrix composed")):
                found_matrix = True
            elif (line.startswith("     A")):
                pass
            elif (found_matrix):
                numbers = line.strip().split()
                if (len(numbers) != 6) :
                    break # bug in output
                if (float(numbers[5]) >= 0.35):
                    found_start = True
                if (found_start and (float(numbers[5]) < 0.35)):
                    found_end = True
                if (found_start and not found_end):
                    max_num = float(numbers[0])
                    max_i = 0
                    for i in range(1,4):
                        if (float(numbers[i]) > max_num):
                            max_num = float(numbers[i])
                            max_i = i
                    SD = SD + lookup[max_i]
                    
    # clean up the files
    #p = Popen("rm " + processed_cds_file, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + rbs_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    p = Popen("rm " + rbs_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    p = Popen("rm " + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    #p = Popen("rm " + profile_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    return(SD)



# expected file
csv_with_file_prefixes = "DiwanDataFilesInfo.csv"
csv_with_tompa_sd_added = 'DiwanDataFiles_WithTompa_no_filter.csv'
input_file_path = "/scratch4/moamin/ncbi_bacteria_db/GbBac_FNA/"
output_file_path = "./"
#file_prefix = "Sulfolobus_acidocaldarius"

count = 0
with open(csv_with_file_prefixes, 'r') as in_file:
    with open(csv_with_tompa_sd_added, 'w') as out_file:
        csvReader = csv.reader(in_file, delimiter=',')
        csvWriter = csv.writer(out_file, delimiter=',')
        for row in csvReader:
            if (row[0] == "name"):
                row.append("tompa_sd")
                csvWriter.writerow(row)
                continue
            count += 1
            if (count > 10):
                break
            file_prefix = row[1]
            #if (file_prefix != "GCA_000946735.1_Vibrio_cholerae_G_298_Guinea"):
             #   continue
            SD = run_Tompa_pipeline(file_prefix, input_file_path, output_file_path)
            if ((len(SD) < 4) or (len(SD) > 6)):
                SD = "------"
            row.append(SD)
            csvWriter.writerow(row)
            
            
# TODO: provide path to the installed tompa algo

GCA_000008885.1_ASM888v1
GCA_000012765.1_ASM1276v1
GCA_000008365.1_ASM836v1
GCA_000007725.1_ASM772v1
GCA_000006625.1_ASM662v1
GCA_000011225.1_ASM1122v1
GCA_000195875.1_ASM19587v1
GCA_000012225.1_ASM1222v1
GCA_000008305.1_ASM830v1
GCA_000007325.1_ASM732v1


In [37]:
from subprocess import Popen, PIPE, STDOUT
import subprocess
import csv
import scipy
from os.path import join
import glob

# expected perl: v5
# expected input files:
# / *_genomic.fna.gz
# keep_only_valid_orfs.pl, tompa_test installed, 

def run_Tompa_pipeline(file_prefix, input_path, output_path): 

    print(file_prefix)
    rbs_input = output_path + file_prefix + '_RBSidentify_input'
    rbs_output = output_path + file_prefix + '_RBSidentify_output'
    profile_input = output_path + file_prefix + '_profile_input'
    profile_output = output_path + file_prefix + '_profile_output' 
    
    # pre-process to get rid of bad orfs
    cds_file = input_path + file_prefix + "_cds_from_genomic.fna.gz"
    processed_cds_file = output_path + file_prefix + "_cds_processed.fa"
    try:
        ret = subprocess.check_output(["perl", "keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing keep_only_valid_orfs for ", cds_file, "; ", exc.returncode, exc.output)
    
    #TODO later make sequence upstream of genes handle multiple chromosomes!!!!    
    full_genome_file = input_path + file_prefix + "_genomic.fna.gz"
    try:
        ret = subprocess.check_output(["perl", "get_sequence_upstream_of_gene.pl", processed_cds_file,\
                                      full_genome_file, rbs_input], stderr=subprocess.STDOUT)
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing get_sequence_upstream_of_gene.pl for ", cds_file, "; ", exc.returncode, exc.output)
    
    
    # run RBSidentify
    p = Popen(['tompa_test/RBSidentify/RBSidentify'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input= (rbs_input + '\n' + rbs_output + '\nn\n7\n1\nn\n').encode())[0]
    
    # check that the output file was created
    names= glob.glob(rbs_output)
    if (len(names) == 0):
        return("") # problem with input
     #   print("Error, more than 1 file with prefix:", first)
    
    # process output of RBSidentify 
    p = Popen("tail -n +27 " + rbs_output  + "|  awk '{print $NF,$0}' | sort -nr | cut -f2- -d' ' >"  + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()

    # run profile
    p = Popen(['tompa_test/profile/profile'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input=(profile_input + '\n' + rbs_input + '\n'  + profile_output + '\n20\n7\n1\nn\n').encode())[0]

    lookup = { 0:'A', 1:'C', 2:'G', 3:'T'}
    found_matrix = False 
    found_start = False
    found_end = False
    
                    
    SDs = []
    SD = ""
    with open(profile_output, "r+") as f:
        last_threshold = 0
        for line in f:
            if (line.startswith("% Weight matrix composed")):
                found_matrix = True
            elif (line.startswith("     A")):
                pass
            elif (found_matrix):
                numbers = line.strip().split("\t")
                if (len(numbers) != 6) :
                    break # continue # bug in output
                if (not found_start and float(numbers[5]) >= 0.4): 
                    found_start = True
                    found_end = False
                    SD = ""
                    #print("found start")
                if (found_start and (float(numbers[5]) < 0.4) and (last_threshold < 0.4)):
                    found_end = True
                    found_start = False
                    #print("found end")
                    SDs.append(SD[:-1])
                if (found_start and not found_end):
                    max_num = float(numbers[0])
                    max_i = 0
                    for i in range(1,4):
                        if (float(numbers[i]) > max_num):
                            max_num = float(numbers[i])
                            max_i = i
                    SD = SD + lookup[max_i]
                last_threshold = float(numbers[5])
    maxSD = ""
    for SD in SDs:
        if(len(SD) > len(maxSD)):
            maxSD = SD              
                    
    # clean up the files
    p = Popen("rm " + processed_cds_file, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    p = Popen("rm " + rbs_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    p = Popen("rm " + rbs_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    #p = Popen("rm " + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + profile_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    return(maxSD)



# expected file
#csv_with_file_prefixes = ""
#csv_with_tompa_sd_added = ''
csv_with_file_prefixes = "all_just_tompa_Bacteria_DataFilesInfo.csv"
csv_with_tompa_sd_added = 'all_just_tompa_Bacteria_DataFilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "166_bad_tompa_Bacteria_DataFilesInfo.csv"
#csv_with_tompa_sd_added = '166_bad_tompa_Bacteria_DataFilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Our_NOT_IN_DIWAN_SD_Bacteria_DataFilesInfo.csv"
#csv_with_tompa_sd_added = 'Our_NOT_IN_DIWAN_SD_Bacteria_DataFilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Diwan_MissingArchaea_FilesInfo.csv"
#csv_with_tompa_sd_added = 'Diwan_MissingArchaea_FilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Diwan_MissingBacteria_FilesInfo.csv"
#csv_with_tompa_sd_added = 'Diwan_MissingBacteria_FilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Ruhul_Annotate_153_DataFilesInfo.csv"
#csv_with_tompa_sd_added = 'Ruhul_Annotate_153_DataFiles_WithTompa_v4.csv'
#csv_with_file_prefixes = "DiwanDataFilesInfo.csv"
#csv_with_tompa_sd_added = 'DiwanDataFiles_WithTompa_v4.csv'
#csv_with_file_prefixes = "Our_SD_Bacteria_DataFilesInfo.csv"
#csv_with_tompa_sd_added = 'Our_SD_Bacteria_DataFilesInfo_with_Tompa.csv'
#input_file_path = "/scratch4/moamin/ncbi_archaea_db/GbArchaea_FNA/"
input_file_path = "/scratch4/moamin/ncbi_bacteria_db/GbBac_FNA/"
output_file_path = "./"
#file_prefix = "Sulfolobus_acidocaldarius"

count = 0
with open(csv_with_file_prefixes, 'r') as in_file:
    with open(csv_with_tompa_sd_added, 'w') as out_file:
        csvReader = csv.reader(in_file, delimiter=',')
        csvWriter = csv.writer(out_file, delimiter=',')
        for row in csvReader:
            if (row[0] == "name"):
                #row.append("tompa_sd")
                csvWriter.writerow(row)
                continue
            count += 1
            #if (count > 2):
             #   break
            file_prefix = row[1]
            #if (file_prefix != "GCA_000282115.1_Chryseobacterium.strCF314_v1.0"):
             #   continue
            SD = run_Tompa_pipeline(file_prefix, input_file_path, output_file_path)
            if (len(SD) < 4): 
                SD = "------"
            if (len(SD) > 6):
                SD = SD[:6]
            #print(row[0])
            #print(SD)
            #print(row[3])
            #row.append(SD)
            if (SD == row[3]):
                csvWriter.writerow(row)
            else:
                new_row = row[:3]
                new_row.append(SD)
                csvWriter.writerow(new_row)
            
# TODO: provide path to the installed tompa algo

GCA_000020605.1_ASM2060v1
GCA_000580335.1_ASM58033v1
GCA_000581715.1_ASM58171v1
GCA_000046845.1_ASM4684v1
GCA_000092025.1_ASM9202v1
GCA_000009365.1_ASM936v1
GCA_000014785.1_ASM1478v1
GCA_000204075.1_ASM20407v1
GCA_000013385.1_ASM1338v1
GCA_000011945.1_ASM1194v1
GCA_000013125.1_ASM1312v1
GCA_900018365.1_IMG-taxon_2615840617_annotated_assembly
GCA_000008625.1_ASM862v1
GCA_000747155.1_ASM74715v1
GCA_000025965.1_ASM2596v1
GCA_000012225.1_ASM1222v1
GCA_000008165.1_ASM816v1
GCA_000307855.1_BAZO1.0
GCA_000307875.1_BABA1.0
GCA_000007825.1_ASM782v1
GCA_000399225.1_Baci_cere_IS195_V1
GCA_000399245.1_Baci_cere_MC118_V1
GCA_000290955.1_Baci_cere_VD022_G8177_V1
GCA_000009825.1_ASM982v1
GCA_000011145.1_ASM1114v1
GCA_000011645.1_ASM1164v1
GCA_001307105.1_ASM130710v1
GCA_000009045.1_ASM904v1
GCA_000940785.1_ASM94078v1
GCA_001304035.1_ASM130403v1
GCA_001552775.1_ASM155277v1
GCA_000025985.1_ASM2598v1
GCA_000699665.1_ASM69966v1
GCA_000011065.1_ASM1106v1
GCA_001567255.1_ASM156725v1
GCA_001525625.1_ASM1525

GCA_000756505.1_ASM75650v1
GCA_001001785.1_ASM100178v1
GCA_000992445.1_ASM99244v1
GCA_000996245.1_ASM99624v1
GCA_001002825.1_ASM100282v1
GCA_001002925.1_ASM100292v1
GCA_000993485.1_ASM99348v1
GCA_000006825.1_ASM682v1
GCA_000011605.1_ASM1160v1
GCA_000012885.1_ASM1288v1
GCA_000932055.1_ASM93205v1
GCA_000196255.1_ASM19625v1
GCA_000196155.1_ASM19615v1
GCA_000013865.1_ASM1386v1
GCA_000007585.1_ASM758v1
GCA_000011465.1_ASM1146v1
GCA_000008345.1_ASM834v1
GCA_000014225.1_ASM1422v1
GCA_000026085.1_ASM2608v1
GCA_000414275.1_Wild_Type_454AllContigs
gzip: /scratch4/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_000414275.1_Wild_Type_454AllContigs_cds_from_genomic.fna.gz: No such file or directory

GCA_001641345.1_ASM164134v1
GCA_000026105.1_ASM2610v1
GCA_000012445.1_ASM1244v1
GCA_000007565.2_ASM756v2
GCA_000012245.1_ASM1224v1
GCA_000012305.1_ASM1230v1
GCA_000013905.1_ASM1390v1
GCA_000203875.1_ASM20387v1
GCA_000009125.1_ASM912v1
GCA_000092045.1_ASM9204v1
GCA_000009265.1_ASM926v1
GCA_000269625.2_PB_Rhod_Spha

In [34]:
print("hi")

hi
